In [ ]:
import subprocess
from pydub import AudioSegment
import math
import os

def extract_audio_from_video(video_path,audio_path):
    command=["ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command,check=True)
    return audio_path

def split_audio_into_chunks(audio_path,chunk_size,chunks_folder):
    audio = AudioSegment.from_mp3(audio_path)
    chunks = math.ceil(len(audio)/chunk_size)
    chunk_size = chunk_size*60*1000
    if not os.path.exists(chunks_folder):
        os.makedirs(chunks_folder)
    for i in range(chunks):
        chunk = audio[i*chunk_size:(i+1)*chunk_size]
        chunk.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")

extract_audio_from_video("files/test.mp4","files/audio.mp3")
split_audio_into_chunks("files/audio.mp3",10,"files/chunks")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --

'files/audio.mp3'

In [ ]:
from pydub import AudioSegment

track = AudioSegment.from_mp3("files/audio.mp3")
track.duration_seconds




2529.674739229025

In [10]:
five_minutes = 5 * 60 * 1000

first_five_minutes = track[:five_minutes]

first_five_minutes.export("files/first_five.mp3", format="mp3")

<_io.BufferedRandom name='files/first_five.mp3'>

In [13]:
import math
ten_minutes = 10 * 60 * 1000
chunks = math.ceil(len(track)/ten_minutes)
for i in range(chunks):
    chunk = track[i*ten_minutes:(i+1)*ten_minutes]
    chunk.export(f"files/chunks/chunk_{i}.mp3", format="mp3")